In [14]:
import pandas as pd
import csv
import datetime
import numpy as np
from statsmodels.api import OLS
import matplotlib.pyplot as plt
import os
import re
import statsmodels.api as sm
from scipy import stats

In [15]:
#处理每个因子的月度收益率
#index='beta'#处理的数据字段
time='time'#对应的时间字段
path2='F:/test/month.csv'
path_return='F:/test/test.csv'

In [16]:
def process_csv(path2,docket,index):
    if (os.path.exists(path2)==False):
        with open(path2,'w') as f:
            f.write('monthly rate input\n')
#path_return为净值的csv
    cache=pd.DataFrame(docket,columns=[index])
    temp=pd.read_csv(path2)
    temp=pd.concat([temp,cache],axis=1)
    messup(temp,path2)
    temp.to_csv(path2,index=False,header=True)
    print('数据计算完成')

def process_month_return(index,time,path_return,path2):
    #path路径为读取保存每日净值文件的csv
    df=pd.read_csv(path_return,encoding="gbk")
    date=len(df.loc[:,[time]])-1
    #处理股指的月收益率
    docket=[]

    value=df.loc[[date],[index]].values[0][0]
    temp=(datetime.datetime.strptime(df.loc[[0],[time]].values[0][0],'%Y/%m/%d %H:%M')).month
    for i in range(0,date,1):

        if (temp!=(datetime.datetime.strptime(df.loc[[i],[time]].values[0][0],'%Y/%m/%d %H:%M')).month):
            docket.append((df.loc[[i],[index]].values[0][0]-value)/value)
            value=df.loc[[i],[index]].values[0][0]

        temp=(datetime.datetime.strptime(df.loc[[i],[time]].values[0][0],'%Y/%m/%d %H:%M')).month

    docket.append((df.loc[[0],[index]].values[0][0]-value)/value) #加上最后一天的月收益率
    
    process_csv(path2,docket,index)
#表格规整
def messup(df,path):
    for i in range(10):
        if 'Unnamed: 0' in df.columns.values:
            df.drop(columns='Unnamed: 0',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0列')
        elif 'Unnamed: 0.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1列')
        elif 'Unnamed: 0.1.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1.1列')
        elif 'Unnamed: 0.1.1.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1.1.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1.1.1列')
        elif 'Unnamed: 0.1.1.1.1' in df.columns.values:
            df.drop(columns='Unnamed: 0.1.1.1.1',axis=1,inplace=True)
            print('已删除多余的Unnamed: 0.1.1.1.1列')
    if (os.path.exists(path)):
        os.remove(path)

In [17]:
if (os.path.exists(path2)):
    os.remove(path2)
    print("原文件删除成功")
else:
    print("原文件不存在，创建新的原文件")

原文件删除成功


In [18]:
#有时时间格式会发生报错，要么换成%Y-%m-%d %H:%M:%S要么换成%Y/%m/%d %H:%M:%S要么根据报错提示换成别的形式
p=pd.read_csv(path_return)
for x in p.columns[1:]:
    process_month_return(x,time,path_return,path2)
print("计算完成")

数据计算完成
数据计算完成
数据计算完成
数据计算完成
数据计算完成
计算完成


In [19]:
#股指数据的选择并保存在本地   股指收益率文件index.csv
#股指选择沪深300或者中证指数  基金收益率文件fund.csv
#沪深300 000300.XSHG   中证700  000907.XSHG
# https://www.joinquant.com/data/dict/indexData
stock='000300.XSHG'
info=get_price(stock, start_date='2009-4-13', end_date='2016-12-31', frequency='daily', fields='open', skip_paused=True, fq='pre')
info.to_csv('F:/test/index.csv')

In [20]:
#东方红基金月收益率计算
#股指月收益率计算
# -*- coding: utf-8 -*-
#df=pd.read_csv(path,encoding="unicode_escape")
df_index=pd.read_csv("F:/test/index.csv",encoding="gbk")
df_fund=pd.read_csv("F:/test/fund.csv",encoding="gbk")

In [21]:
index='fuquan'#处理的数据字段
time='riqi'#对应的时间字段

date=len(df_fund.loc[:,[time]])-1
#处理股指的月收益率
docketfund=[]

value=df_fund.loc[[date],[index]].values[0][0]
temp=(datetime.datetime.strptime(df_fund.loc[[date],[time]].values[0][0],'%Y/%m/%d')).month
for i in range(date,0,-1):
        
    if (temp!=(datetime.datetime.strptime(df_fund.loc[[i],[time]].values[0][0],'%Y/%m/%d')).month):
        docketfund.append((df_fund.loc[[i],[index]].values[0][0]-value)/value)
        value=df_fund.loc[[i],[index]].values[0][0]
        
    temp=(datetime.datetime.strptime(df_fund.loc[[i],[time]].values[0][0],'%Y/%m/%d')).month

docketfund.append((df_fund.loc[[0],[index]].values[0][0]-value)/value) #加上最后一天的月收益率

process_csv(path2,docketfund,index)

数据计算完成


In [22]:
index='MKT'#处理的数据字段
time='time'#对应的时间字段
#从平台上直接读取的数据有问题，需要索引规整
df_index=df_index.rename(columns={'Unnamed: 0':'time','open':'MKT'})
date=len(df_index.loc[:,[time]])-1
#处理股指的月收益率
docketindex=[]

value=df_index.loc[[date],[index]].values[0][0]
temp=(datetime.datetime.strptime(df_index.loc[[0],[time]].values[0][0],'%Y-%m-%d')).month
for i in range(0,date,1):
        
    if (temp!=(datetime.datetime.strptime(df_index.loc[[i],[time]].values[0][0],'%Y-%m-%d')).month):
        docketindex.append((df_index.loc[[i],[index]].values[0][0]-value)/value)
        value=df_index.loc[[i],[index]].values[0][0]
        
    temp=(datetime.datetime.strptime(df_index.loc[[i],[time]].values[0][0],'%Y-%m-%d')).month

docketindex.append((df_index.loc[[0],[index]].values[0][0]-value)/value) #加上最后一天的月收益率

process_csv(path2,docketindex,index)

数据计算完成


In [23]:
#得到docketindex[]和docketfund[]这两个列表，包含了在这一时间段内指数和fund收益率的数据
# docketfund=np.array(docketfund,dtype=np.float)
# docketindex=np.array(docketindex,dtype=np.float)

# docketfund=pd.Series(docketfund)
# docketindex=pd.Series(docketindex)

# docketfund.index = pd.Index(sm.tsa.datetools.dates_from_range('1',str(len(docketfund))))

In [24]:
#计算超额收益率的部分

In [25]:
#设定年化率4%，在此情况下选用月收益率作为无风险利率
r=pow(1.04,12)-1

In [26]:
#生成premium和时间索引两列
df=pd.read_csv(path2)

In [27]:
df['t']=list(range(1,len(df)+1))#生成时间
df['premium']=df.apply(lambda x: x['fuquan'] - 0.01*r, axis=1)#生成超额收益

In [28]:
messup(df,path2)
df.to_csv(path2,index=False,header=True)#重新存取到csv文件

In [181]:
#一元线性回归部分，确定指数
#综合比较沪深300、中证500、中证800、上证50，显然沪深300的相关性最好
time='time'#对应的时间字段
path_month='F:/test/month.csv'

In [182]:
x=pd.read_csv(path_month)

In [184]:
model=sm.OLS(list(x.fuquan.values),list(x.MKT.values))
reg=model.fit()

In [185]:
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     107.2
Date:                Wed, 06 Nov 2019   Prob (F-statistic):           4.11e-17
Time:                        16:38:27   Log-Likelihood:                 153.75
No. Observations:                  93   AIC:                            -305.5
Df Residuals:                      92   BIC:                            -303.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.5612      0.054     10.355      0.0